In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings

import requests 
import json

warnings.filterwarnings('ignore')
%matplotlib inline

import dtale

MessageError: ignored

In [ ]:
# index des lignes dupliquées (on drop les unhashable)
duplicateRowsDF = df[df.drop(columns=["cordonnees_bus_geometrie", "coordonnees_GPS_WG84"]).duplicated()]

# drop des lignes dupliquées par index
df = df.drop(duplicateRowsDF.index, axis=0)
df.shape

In [ ]:
df.isna().sum()

df = df.dropna()

df.isna().sum().sum()

In [ ]:
q1 = df["ecart_horaire_en_secondes"].quantile(0.05)
q3 = df["ecart_horaire_en_secondes"].quantile(0.95)
iqr = q3 - q1

df["ecart_horaire_en_secondes"] = df["ecart_horaire_en_secondes"].clip(lower=q1, upper=q3)

In [ ]:
df[['latitude', 'longitude']] = df['coordonnees_GPS_WG84'].str.split(',', expand=True)

cols = ['latitude', 'longitude']
for col in cols :
    df[col] = df[col].map(lambda x: str(x).lstrip('[').rstrip(']')).astype(float)

df["year"] = df["horodatage"].dt.year
df["month"] = df["horodatage"].dt.month
df["day"] =  df["horodatage"].dt.day
df["hours"] = df["horodatage"].dt.hour
df["date"] = pd.to_datetime(df[["year", "month", "day"]])


df["jour_semaine"] = df["date"].dt.day_name().map({"Monday": "Lundi", "Tuesday": "Mardi", "Wednesday": "Mercredi", "Thursday": "Jeudi", "Friday": "Vendredi", "Saturday": "Samedi", "Sunday": "Dimanche"})
df.drop(columns=["date"], inplace=True)

In [ ]:
df.columns

In [ ]:
# on transforme l'heure estimee de passage à l 'arret  : 
df["year_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.year
df["month_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.month
df["day_estimation_passage_arret"] =  df["Heure_estimee_de_passage_a_L_arret"].dt.day
df["hours_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.hour
df["minutes_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.minute
df["secondes_estimation_passage_arret"] = df["Heure_estimee_de_passage_a_L_arret"].dt.second
df.drop(columns=["Heure_estimee_de_passage_a_L_arret"], inplace=True)



In [ ]:
# dummy 
df = pd.get_dummies(df)